### 사이킷런:
ML 모델 학습을 위해서 fit()
학습된 모델의 예측을 위해 predict() 메소드 제공

-분류알고리즘
Classifier class\
-회귀 알고리즘
Regressor class\
-두가지 합치면
[Estimator class] 인자로 받음\
cross_val_score() : evaluation 함수\
GridSearchCV : 하이퍼 파라미터 튜닝

cross_val_score()\
GridSearchCV.fit()\
fit()\
predict()\
호출

[실습 프로젝트]
1. 데이터준비
2. 학습알고리즘 선택
3. 학습 데이터를 통한 모델학습
4. 테스트 데이터를 통한 모델 검증

[sk-learn API ref]


## 텍스트 분류실습-뉴스그룹 분류개요

https://dschloe.github.io/python/nlp/ch03_news_group_classification/

In [1]:
# 텍스트 정규화
# 데이터: 사이킷런 제공 fetch_20newsgroups() 이용

In [2]:
from sklearn.datasets import fetch_20newsgroups

In [3]:
news_data = fetch_20newsgroups(subset='all', random_state=156)
# subset : 안에 어떤 데이터 가져올건지 설정
# random_state: 수행시마다 동일한 결과를 얻기위해 적용; replay 가능하게함
# random 함수의 seed값
# 컴퓨터는 랜덤하게 한다는것 자체가 어려움, 그 안에 또 규칙이 있음
print(news_data.keys())

dict_keys(['data', 'filenames', 'target_names', 'target', 'DESCR'])


In [4]:
import pandas as pd
print('target 클래스의 값과 분포도 \n', pd.Series(news_data.target).value_counts().sort_index())
print('target 클래스의 이름들 \n', news_data.target_names)
print(len(news_data.target_names)) # target 20개 (0~19)

target 클래스의 값과 분포도 
 0     799
1     973
2     985
3     982
4     963
5     988
6     975
7     990
8     996
9     994
10    999
11    991
12    984
13    990
14    987
15    997
16    910
17    940
18    775
19    628
dtype: int64
target 클래스의 이름들 
 ['alt.atheism', 'comp.graphics', 'comp.os.ms-windows.misc', 'comp.sys.ibm.pc.hardware', 'comp.sys.mac.hardware', 'comp.windows.x', 'misc.forsale', 'rec.autos', 'rec.motorcycles', 'rec.sport.baseball', 'rec.sport.hockey', 'sci.crypt', 'sci.electronics', 'sci.med', 'sci.space', 'soc.religion.christian', 'talk.politics.guns', 'talk.politics.mideast', 'talk.politics.misc', 'talk.religion.misc']
20


In [5]:
print(news_data.data[1]) # 뉴스그룹제목, 작성자, 소속 등 다양한 정보 가짐
# 불필요한 부분 remove 파라미터로 제거

From: jlevine@rd.hydro.on.ca (Jody Levine)
Subject: Re: insect impacts
Organization: Ontario Hydro - Research Division
Lines: 64

I feel childish.

In article <1ppvds$92a@seven-up.East.Sun.COM> egreen@East.Sun.COM writes:
>In article 7290@rd.hydro.on.ca, jlevine@rd.hydro.on.ca (Jody Levine) writes:
>>>>
>>>>how _do_ the helmetless do it?
>>>
>>>Um, the same way people do it on 
>>>horseback
>>
>>not as fast, and they would probably enjoy eating bugs, anyway
>
>Every bit as fast as a dirtbike, in the right terrain.  And we eat
>flies, thank you.

Who mentioned dirtbikes? We're talking highway speeds here. If you go 70mph
on your dirtbike then feel free to contribute.

>>>jeeps
>>
>>you're *supposed* to keep the windscreen up
>
>then why does it go down?

Because it wouldn't be a Jeep if it didn't. A friend of mine just bought one
and it has more warning stickers than those little 4-wheelers (I guess that's
becuase it's a big 4 wheeler). Anyway, it's written in about ten places that
the 

In [6]:
# remove 파라미터 이용해 불필요한것 제거
# 훈련 데이터/테스트 데이터로 분류 -> 이미 나뉘어진것 가져오기

# subset='train'으로 학습용 데이터만 추출
# remove=('headers', 'footers', 'quotes')로 내용만 추출
train_news = fetch_20newsgroups(subset='train', remove=('headers', 'footers', 'quotes'), random_state=156)

X_train = train_news.data
y_train = train_news.target

# subset='test'으로 테스트 데이터만 추출
# remove=('headers', 'footers', 'quotes')로 내용만 추출
test_news = fetch_20newsgroups(subset='test', remove=('headers', 'footers', 'quotes'), random_state=156)

X_test = test_news.data
y_test = test_news.target

print('학습 데이터 크기 {0}, 테스트 데이터 크기 {1}'.format(len(train_news.data), len(test_news.data)))

학습 데이터 크기 11314, 테스트 데이터 크기 7532


## Countvector

단어 피처에 값을 부여할 때, 각 문서에서 해당 단어가 나타나는 횟수, 즉 Count를 부여하는 경우를 카운트 벡터화\
카운트 벡터화에서는 값이 높을수록 중요한 단어로 인식\
카운트 기반 벡터화는 카운트 값이 높을수록 중요한 단어로 인식한다고 했습니다. 하지만 단순히 단어의 빈도만 고려한다면 모든 문서에서 자주 쓰일 수밖에 없는 단어들이 (불용어 등) 중요하다고 인식될 수 있습니다. 가령 A문서에서도 'The'가 가장 많이 등장하고, B문서에서도 'The'가 가장 많이 등장한다고 해서 두 문서가 유사한 문서라고 판단할 수는 없습니다. 이런 문제를 보완하기 위해 TF-IDF(Term Frequency - Inverse Document Frequency) 벡터화를 사용

https://bkshin.tistory.com/entry/NLP-6-카운터-기반-벡터화CountVectorizer와-TF-IDF-벡터화

In [7]:
# 피처 벡터화 변환: CountVectorizer 이용해 학습데이터 텍스트를 피쳐벡터화
'''
CountVectorizer
: 단어들의 카운트(출현 빈도(frequency))로 여러 문서들을 벡터화
카운트 행렬, 단어 문서 행렬 (Term-Document Matrix, TDM))
모두 소문자로 변환시키기 때문에 me 와 Me 는 모두 같은 특성이 된다.
https://wikidocs.net/33661

문서집합 -> 단어토큰 생성 -> 단어수 셈 -> BOW 인코딩 벡터 만듬
'''

from sklearn.feature_extraction.text import CountVectorizer

# count vectorization 피처 벡터화 변환진행
cnt_vect = CountVectorizer()
cnt_vect.fit(X_train) # 실행 countvectorizer


CountVectorizer()

In [8]:
cnt_vect.vocabulary_ # freq; 빈돗수 높을수록 중요한 단어라고 인식

{'what': 96391,
 'did': 33551,
 'not': 66511,
 'get': 43217,
 'with': 96917,
 'my': 64435,
 'drive': 35147,
 'cd300i': 26277,
 'is': 51136,
 'the': 88532,
 'system': 87099,
 'install': 50300,
 'cd': 26271,
 'you': 100208,
 'listed': 56936,
 'as': 19756,
 'any': 18903,
 'ideas': 48659,
 'about': 16251,
 'how': 47460,
 'can': 25590,
 'one': 68102,
 'bought': 23790,
 'iivx': 48926,
 '120': 2223,
 'from': 41715,
 'direct': 33799,
 'express': 38824,
 'in': 49447,
 'chicago': 26992,
 'no': 66242,
 'complaints': 28805,
 'at': 20123,
 'all': 17936,
 'good': 43961,
 'price': 73151,
 'service': 81774,
 'btw': 24444,
 've': 93870,
 'heard': 46088,
 'that': 88519,
 'be': 21987,
 'used': 92875,
 'to': 89360,
 'boot': 23663,
 'mac': 58921,
 'however': 47469,
 'will': 96683,
 'accept': 16406,
 'caddy': 25370,
 'machine': 58962,
 'off': 67683,
 'it': 51326,
 'then': 88587,
 'dave': 31939,
 'and': 18521,
 'day': 31990,
 'ask': 19853,
 'me': 60560,
 'question': 75283,
 'truly': 90374,
 'say': 80633,
 'i

In [9]:
X_train_cnt_vect = cnt_vect.transform(X_train) #변환

In [10]:
X_test_cnn_vect = cnt_vect.transform(X_test)

In [11]:
# ML 학습/예측/평가
# 로지스틱회귀 활용 (binary classification, sigmoid, 0 OR 1 ouput)
# data의 특징활용

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# logitstic regression 이용 학습/예측/평가 수행
lr_clf = LogisticRegression()
lr_clf.fit(X_train_cnt_vect, y_train) # train (data-target)
pred = lr_clf.predict(X_test_cnn_vect) # test data가지고 예측

print('CountVectorized Logistic Regression의 예측 정확도는 {0:.3f}'.format(accuracy_score(y_test, pred)))

# stopwords 처리 안함

CountVectorized Logistic Regression의 예측 정확도는 0.607


/Users/young/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


## tf-idf 클래스로 bow 만들기

TF-IDF는 개별 문서에서 자주 등장하는 단어에 높은 가중치를 주되, 모든 문서에서 전반적으로 자주 등장하는 단어에 대해서는 패널티를페널티를 주는 방식으로 값을 부여
-> 해당단어가 실질적으로 중요한 단어인지 검사

- 문서양 많을경우: 일반적으로 카운트 기반 벡터화보다 tf-idf 방식의 벡터화 사용 많이함

https://bkshin.tistory.com/entry/NLP-6-카운터-기반-벡터화CountVectorizer와-TF-IDF-벡터화

In [12]:
# count -> tf-idf 기반으로 벡터화 변경해서 예측 모델 수행

from sklearn.feature_extraction.text import TfidfVectorizer

# TF-IDF 벡터화를 적용하여 학습 데이터 세트와 테스트 데이터 세트 변환. 
tfidf_vect = TfidfVectorizer()
tfidf_vect.fit(X_train)

X_train_tfidf_vect = tfidf_vect.transform(X_train)
X_test_tfidf_vect = tfidf_vect.transform(X_test)

In [13]:
sorted(tfidf_vect.vocabulary_.items()) # index 와 단어 할당 확인

[('00', 0),
 ('000', 1),
 ('0000', 2),
 ('00000', 3),
 ('000000', 4),
 ('00000000', 5),
 ('0000000004', 6),
 ('00000000b', 7),
 ('00000001', 8),
 ('00000001b', 9),
 ('00000010', 10),
 ('00000010b', 11),
 ('00000011', 12),
 ('00000011b', 13),
 ('00000074', 14),
 ('00000093', 15),
 ('000000e5', 16),
 ('00000100', 17),
 ('00000100b', 18),
 ('00000101', 19),
 ('00000101b', 20),
 ('00000110', 21),
 ('00000110b', 22),
 ('00000111', 23),
 ('00000111b', 24),
 ('000005102000', 25),
 ('00000510200001', 26),
 ('00000ee5', 27),
 ('00001000', 28),
 ('00001000b', 29),
 ('00001001', 30),
 ('00001001b', 31),
 ('00001010', 32),
 ('00001010b', 33),
 ('00001011', 34),
 ('00001011b', 35),
 ('000010af', 36),
 ('00001100', 37),
 ('00001100b', 38),
 ('00001101', 39),
 ('00001101b', 40),
 ('00001110', 41),
 ('00001110b', 42),
 ('00001111', 43),
 ('00001111b', 44),
 ('000042', 45),
 ('000062david42', 46),
 ('000094', 47),
 ('0001', 48),
 ('00010000', 49),
 ('00010000b', 50),
 ('00010001', 51),
 ('00010001b', 5

In [14]:
lr_clf = LogisticRegression()
lr_clf.fit(X_train_tfidf_vect, y_train)
pred = lr_clf.predict(X_test_tfidf_vect)

print("TF-IDF Logistic Regression의 예측 정확도는 {0:.3f}".format(accuracy_score(y_test, pred)))

TF-IDF Logistic Regression의 예측 정확도는 0.674


TF-IDF가 단순 카운트 기반보다 더 높은 예측 정확도 제공
0.674 > 0.607

In [15]:
# 모형 업그레이드 step1

# 모델 업그레이드 -> 최상의 피처 전처리 수행 필요 (전처리)

# stop words 필터링 추가 & ngram을 기본 (1, 1)에서 (1, 2); uni & bigram 로 변경해 피처 벡터화 적용 ; 컨텍스트

tfidf_vect = TfidfVectorizer(stop_words="english", ngram_range=(1,2), max_df=300) # max_df 최대 문서의 수 빈도값
tfidf_vect.fit(X_train)

X_train_tfidf_vect = tfidf_vect.transform(X_train)
X_test_tfidf_vect = tfidf_vect.transform(X_test)

lr_clf = LogisticRegression()
lr_clf.fit(X_train_tfidf_vect, y_train)
pred = lr_clf.predict(X_test_tfidf_vect)

print("TF-IDF Logistic Regression의 예측 정확도는 {0:.3f}".format(accuracy_score(y_test, pred)))



TF-IDF Logistic Regression의 예측 정확도는 0.692


In [16]:
# 모형 업그레이드 2단계

# 로지스틱 회귀의 하이퍼 파라미터 최적화 수행 -> GridSearchCV 사용 (GridSerach + Cross Validation)
# 가장 최적의 파라미터값 찾아주는 방법 중 하나
# 1) 무식하게 다 해보기; 파라피터 후보값 사전 정의, 반복문으로 다 적용 ; 2) 추가적 교차검증 수행

from sklearn.model_selection import GridSearchCV
import time 
import datetime

start = time.time()

# 최적 C값 도출 튜닝 수행 / 과적합 방지용
params = {'C' : [0.01, 0.1]} # [0.01, 0.1, 1, 5, 10] # 하이퍼파라미터 세트 dict 형태로 변수 저장; C = alpha
# c는 규제강도를 조정하는 alpha 값 역수 (c = 1/alpha), 즉 c가 클수록 규제강도가 큼
# 추가 penalty l1, l2
# l1 = 벡터 각 성분의 길이를 나타낸 값 (2,3) -> 2+3 = 5 로 계산
# l2 = 이동거리를 유클리드거리로 계산 2의 제곱+3의 제곱 = 13에 루트값
# 제곱값이 무한히 커져서 오버피팅이 될 위험을 줄여주기 위한 작업;  

grid_cv_lr = GridSearchCV(lr_clf, param_grid=params, cv=2, scoring="accuracy", verbose=1) # cv: 교차 검증에서 몇개로 분할되는지
grid_cv_lr.fit(X_train_tfidf_vect, y_train)
print('Logistic Regression best C parameter :', grid_cv_lr.best_params_)
# print('Logistic Regression Best C Parameter :', grid_cv_lr.best_params_)

sec = time.time()-start
times = str(datetime.timedelta(seconds=sec)).split(".")
times = times[0]
print(times)

Fitting 2 folds for each of 2 candidates, totalling 4 fits
Logistic Regression best C parameter : {'C': 0.1}
0:05:52


In [18]:
# 참고

import time
import datetime
def bench_mark(start):
    sec = time.time() - start
    times = str(datetime.timedelta(seconds=sec)).split(".")
    times = times[0]
    print(times)

In [19]:
# 최적 C값으로 학습된 grid_cv로 예측 및 정확도 평가
pred = grid_cv_lr.predict(X_test_tfidf_vect)
print('TF-IDF Vectorized Logistic Regression의 예측 정확도는 {0:.3f}'.format(accuracy_score(y_test, pred)))

TF-IDF Vectorized Logistic Regression의 예측 정확도는 0.645


### 파이프라인을 통해 ML

In [ ]:
from sklearn.pipeline import Pipeline


start = time.time()

# TfidfVectorizer 객체를 tfidf_vect로, LogisticRegression 객체를 lr_clf로 생성하는 Pipeline 생성

pipeline = Pipeline([
    ('tfidf_vect', TfidfVectorizer(stop_words='english')),
    ('lr_clf', LogisticRegression())
])

# Pipeline에 기술된 각각의 객체 변수에 언더바(_)2개를 연달아 붙여 GridSearchCV에 사용될 
# 파라미터/하이퍼 파라미터 이름과 값을 설정. . 
params = { 'tfidf_vect__ngram_range': [(1,1), (1,2), (1,3)],
           'tfidf_vect__max_df': [100, 300, 700],
           'lr_clf__C': [1,5,10]
}

# GridSearchCV의 생성자에 Estimator가 아닌 Pipeline 객체 입력
grid_cv_pipe = GridSearchCV(pipeline, param_grid=params, cv=2 , scoring='accuracy', verbose=1)
grid_cv_pipe.fit(X_train, y_train)
print(grid_cv_pipe.best_params_ , grid_cv_pipe.best_score_)

pred = grid_cv_pipe.predict(X_test)
print('Pipeline을 통한 Logistic Regression 의 예측 정확도는 {0:.3f}'.format(accuracy_score(y_test ,pred)))

bench_mark(start)


Fitting 2 folds for each of 27 candidates, totalling 54 fits


/Users/young/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Users/young/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logis